<a href="https://colab.research.google.com/github/simionattovini/Hoisting-and-Rigging-Machinery/blob/main/Pendulum%20Load%20Traversing/MET_Sistema_com_Carga_Pendular_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sistema com Carga Pendular - Controle

Para realizar o controle do sistema com carga pendular, podemos partir de duas abordagens equivalentes. A primeira seria imaginar um deslocamento virtual $y$, que estaria puxando o carrinho através de um cabo com rigidez e amortecimento em paralelos, de coeficientes $k_c$ e $c_c$.

Neste caso, $y(t)$ passaria a ser a entrada do sistema, e as equações do sistema se tornariam:

$$
\left[\begin{array}{cc}
I_g + ML^2 & ML \\ ML & M+m
\end{array}
\right]
\left\{
\begin{array}{c}
\ddot{\theta} \\ \ddot{x}
\end{array}
\right\}
+
\left[\begin{array}{cc}
0 & 0 \\ 0 & c_c
\end{array}
\right]
\left\{
\begin{array}{c}
\dot{\theta} \\ \dot{x}
\end{array}
\right\}
+
\left[\begin{array}{cc}
MgL & 0 \\ 0 & k_c
\end{array}
\right]
\left\{
\begin{array}{c}
\theta \\ x
\end{array}
\right\}
=
\left\{
\begin{array}{c}
0 \\ c_c\dot{y}(t) + k_c y(t)
\end{array}
\right\}
$$

Uma forma alternativa, mais próxima à filosofia de controle, seria designar o sistema original na _forma completa_ de estados para controle. Nesta forma, o sistema é representado por:

$$
\left\{
\begin{array}{l}
\{\dot{z}\} = [A]\{z\} + [B]\{u(t)\} \\
\{w\} = [C]\{z\} + [D]\{u(t)\}
\end{array}
\right.
$$

Neste caso, $\{z\}$ é o vetor de estados, $\{u(t)\}$ o vetor de entradas e $\{w\}$ o vetor de saídas. Além disso, $[A]$ é a matriz do sistema, $[B]$ é a matriz de entrada, $[C]$ é a matriz de saída e $[D]$ é a matriz de retroalimentação. Destas, a única matriz que depende do sistema e o representa é a matriz $[A]$. As matrizes $[B]$, $[C]$ e $[D]$ podem ser definidas arbitariamente.

Como vamos começar colocando o sistema original na forma de estados, partiremos da equação:

$$
\underbrace{
\left[\begin{array}{cc}
I_g + ML^2 & ML \\ ML & M+m
\end{array}
\right]}_{M_I}
\left\{
\begin{array}{c}
\ddot{\theta} \\ \ddot{x}
\end{array}
\right\}
+
\underbrace{
\left[\begin{array}{cc}
MgL & 0 \\ 0 & 0
\end{array}
\right]}_{M_K}
\left\{
\begin{array}{c}
\theta \\ x
\end{array}
\right\}
=
\left\{
\begin{array}{c}
0 \\ f(t)
\end{array}
\right\}
$$

Assim a matriz do sistema fica na forma:
$$
[A] =
\left[
\begin{array}{cc}
[0]_{2\times 2} & [I]_{2\times 2} \\
-[M_I]^{-1}[M_K] & [0]_{2\times 2}
\end{array}
\right]
$$

Já a matriz de entrada é escrita na forma:

$$
[B] = \left[
\begin{array}{c}
[0]_{2\times 2} \\ [M_I]^{-1}
\end{array}
\right]
\left[
\begin{array}{c}
0 \\ 1
\end{array}
\right] f(t)
=
\left[
\begin{array}{c}
0 \\ 0 \\ [M_I]^{-1} \left[
\begin{array}{c}
0 \\ 1
\end{array}
\right]
\end{array}
\right]_{4\times 1}
f(t)
$$

A saída deste sistema deve ser apenas a função $x(t)$, portanto a matriz de saída vale:

$$
[C] = [0 \,\, 1 \,\,  0 \,\, 0]
$$

E a de retroalimentação vale:
$$
[D] = 0
$$

Assim, o sistema já possui as 4 matrizes da forma de estudo. Assim, é possível representá-lo programaticamente:

In [ ]:
# Dados do sistema:

M = 200   # kg
m = 10    # kg
L = 10    # m
Ig = 3    # kg.m²
g = 9.81  # m/s²

# Dados do cabo ou controlador:

kc = 100
cc = 10



In [ ]:
import numpy as np
import importlib
spec = importlib.util.find_spec('control')
if spec is None:
    !pip install control

from control import StateSpace, tf

MI = np.asarray([[Ig+M*L**2, M*L], [M*L, M+m]])
MK = np.asarray([[M*g*L, 0], [0, 0]])

A = np.append(np.append(np.zeros((2, 2)), np.eye(2), 1),
              np.append(-np.linalg.inv(MI) @ MK, np.zeros((2, 2)), 1),
              0)

B = np.append(np.zeros((2, 1)), np.linalg.inv(MI) @ np.asarray([[0], [1]]), 0)

C = np.asarray([[0, 1, 0, 0]])

D = np.asarray([[0]])

sys = StateSpace(A, B, C, D)

Podemos controlar este sistema com um controlador PD com função de transferência:

$$
K_s(s) = c_c s + k_c
$$

Agora basta multiplicar a planta do controlador pela planta do sistema e fechar a malha, conforme vimos na aula anterior:

In [ ]:
from control import feedback

s = tf('s')

Ks = kc + cc*s

print(Ks)

Hs = tf(sys)

# Removendo coeficientes fruto de erro numérico
aux = list(Hs.num[0][0])
aux = [el if np.abs(el) > 1e-6 else 0 for el in aux]
while aux[0] == 0:
  aux.pop(0)
Hs.num = [[np.asarray(aux)]]

# Sistema em Malha Fechada
SMF = feedback(Ks*Hs, 1)

<TransferFunction>: sys[341]
Inputs (1): ['u[0]']
Outputs (1): ['y[0]']


10 s + 100
----------
    1



Podemos verificar que são os mesmos sistemas através da comparação dos polos e zeros:


In [ ]:
# Do sistema em que aplicamos a estratégia de controle manuamente:

MCii = np.asarray([[0, 0], [0, cc]])
MKii = np.asarray([[M*g*L, 0], [0, kc]])

# Para o cálculo dos autovalores, a matriz A (no caso, Am) é suficiente.
Am = np.append(np.append(np.zeros((2, 2)),
                         np.eye(2), 1),
               np.append(-np.linalg.inv(MI) @ MKii,
                         -np.linalg.inv(MI) @ MCii, 1),
               0)

# Sistema com Malha Fechada manualmente
SMFm = tf(StateSpace(Am, B, C, D))

# Removendo coeficientes fruto de erro numérico
aux = list(SMFm.num[0][0])
aux = [el if np.abs(el) > 1e-6 else 0 for el in aux]
while aux[0] == 0:
  aux.pop(0)
SMFm.num = [[np.asarray(aux)]]

from control import damp

print('Autovalores do sistema em malha fechada original:')
_ = damp(SMF)
print('\n','Autovalores do sistema com a malha fechada manualmente:',sep='')
_ = damp(SMFm)


Autovalores do sistema em malha fechada original:
    Eigenvalue (pole)       Damping     Frequency
   -0.4875     +5.47j       0.08877         5.492
   -0.4875     -5.47j       0.08877         5.492
  -0.01097   +0.5693j       0.01927        0.5694
  -0.01097   -0.5693j       0.01927        0.5694

Autovalores do sistema com a malha fechada manualmente:
    Eigenvalue (pole)       Damping     Frequency
   -0.4875     +5.47j       0.08877         5.492
   -0.4875     -5.47j       0.08877         5.492
  -0.01097   +0.5693j       0.01927        0.5694
  -0.01097   -0.5693j       0.01927        0.5694


Nosso objetivo é ajustar os valores de $k_c$ e $c_c$ para fazer com que o sistema chegue o mais próximo possível do criticamente amortecido em pelo menos um dos modos de vibração.